**bold text**# Overview: Analysis on labeled dataset provided by SASB

Our initial attempts to train different classifiers to predict the relevant GICs on 10-K disclosures did not yield adequate metrics. The highest level of precision/recall we were able to obtain was around 0.82.

We reviewed the prediction results and the labeled data in details. We realized that there were inconsistency in the labeled data in that:
* Identical 10-K disclosures were labeled with different targets (e.g. not relevant vs. GIC 310)
* Certain 10-K disclosures were labeled incorrectly and/or inconsistently (e.g. collective bargaining, executive compensations, etc.)

As such, we decided to pursue this analysis of the labeled dataset to find a way to help us come up with better and more consistent targets that are aligned with our objectives.

The purpose of this notebook is to:
* Understand how the targets from the labeled datasets were originally determined
* Identify an approach to come up with rules to identify relevant disclosure paragraphs
* Test our approach and present our findings

---
# Preprocessing of the labeled dataset

## Step 1: Import and process labeled data in paragraph form
The purpose of this section is to:
* Import the labeled data, which is already in paragraph form
* Process the labeled data by lemmatizing the words, removing stop words, keeping only alphabets and removing all words with entity types as recognized by spaCy (e.g. amounts, dates, country, names, etc.)
* Export the processed data into csv for future use

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import re
import spacy
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas(desc="progress bar")

# nlp = spacy.load("en_core_web_sm", disable=["tagger", "parser", "ner", "textcat"])
nlp = spacy.load("en_core_web_sm")
# nlp.tokenizer = custom_tokenizer(nlp)

# import data from json
df_300 = pd.read_json('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/labeled_10k/di_hc_rel_train.json', orient=str)

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
# identify relevant GICs
df_300['relevance'] = df_300['relevance_assessment'].apply(lambda x: 1 if x == 'Relevant' else 0)
df_300['gic_code'] = df_300['disclosure_topic_id'].str[-4:-1].astype('int')*df_300['relevance']

# keep relevant columns
df_300 = df_300[['excerpt_id', 'excerpt', 'gic_code']] 

# clean excerpt
# df_300['excerpt'] = df_300['excerpt'].apply(lambda x: re.sub(r'[^\w\s]|\n|\d', ' ', x).lower())
df_300['excerpt_lemma'] = df_300['excerpt'].progress_apply(
    lambda x: ' '.join([t.lemma_.lower() 
    for t in nlp(x) if not t.is_stop and t.is_alpha and t.ent_type_ == '']))

df_300.to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.7 visualized/labeled_data/di_hc_rel_train_lemmatized_no_ent_paragraph.csv', index=False)

# check the distribution of GICs
df_300['gic_code'].value_counts()

0      11463
330     4466
320     2609
310     2088
Name: gic_code, dtype: int64

## Step 2: Import and process labeled data in sentence form
The purpose of this section is to:
* Import the labeled data
* Break the paragraphs into sentences
* Process the sentences by lemmatizing the words, removing stop words, keeping only alphabets and removing all words with entity types as recognized by spaCy (e.g. amounts, dates, country, names, etc.)
* Export the processed data into csv for future use

In [ ]:
import pandas as pd
import spacy
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas(desc="progress bar")

nlp = spacy.load("en_core_web_sm")

# import data from json
df_300 = pd.read_json('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/labeled_10k/di_hc_rel_train.json')

# identify relevant GICs
df_300['relevance'] = df_300['relevance_assessment'].apply(lambda x: 1 if x == 'Relevant' else 0)
df_300['gic_code'] = df_300['disclosure_topic_id'].str[-4:-1].astype('int')*df_300['relevance']

# keep relevant columns
df_300 = df_300[['excerpt_id', 'excerpt', 'gic_code']]

def tokenize_excerpts(df):

  result = pd.DataFrame()

  for idx, r in df.iterrows():
    if (idx+1) % 100 == 0:
      print('processing '+str(idx+1)+'/'+str(df.shape[0]))
    doc = nlp(r['excerpt'])
    original_sents = []
    tokenized_sents = []
    tokenized_ids = []
    sent_id = 0
    for sent in doc.sents:
      s = []
      tokenized_ids.append(r['excerpt_id']+'-'+str(sent_id))
      sent_id += 1
      for t in sent:
        if not t.is_stop and t.is_alpha and t.ent_type_ == '':
          s.append(t.lemma_.lower())
      original_sents.append(sent.text)
      tokenized_sents.append(' '.join(s))
    
    df_tmp = pd.DataFrame()
    df_tmp['excerpt_lemma'] = tokenized_sents
    df_tmp['excerpt'] = original_sents
    df_tmp['excerpt_id'] = tokenized_ids
    df_tmp['gic_code'] = r['gic_code']

    result = pd.concat([result, df_tmp])

  return result

df_300 = tokenize_excerpts(df_300)
df_300.head(10)

# save lemmatized labeled data
# df_300.to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.7 visualized/labeled_data/di_hc_rel_train_lemmatized_no_ent_sent.csv', index=False)


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


processing 100/20626
processing 200/20626
processing 300/20626
processing 400/20626
processing 500/20626
processing 600/20626
processing 700/20626
processing 800/20626
processing 900/20626
processing 1000/20626
processing 1100/20626
processing 1200/20626
processing 1300/20626
processing 1400/20626
processing 1500/20626
processing 1600/20626
processing 1700/20626
processing 1800/20626
processing 1900/20626
processing 2000/20626
processing 2100/20626
processing 2200/20626
processing 2300/20626
processing 2400/20626
processing 2500/20626
processing 2600/20626
processing 2700/20626
processing 2800/20626
processing 2900/20626
processing 3000/20626
processing 3100/20626
processing 3200/20626
processing 3300/20626
processing 3400/20626
processing 3500/20626
processing 3600/20626
processing 3700/20626
processing 3800/20626
processing 3900/20626
processing 4000/20626
processing 4100/20626
processing 4200/20626
processing 4300/20626
processing 4400/20626
processing 4500/20626
processing 4600/206

,excerpt_lemma,excerpt,excerpt_id,gic_code
0,reclassification aoci follow,Reclassifications out of AOCI for the years en...,2017-AAL-TR-AL-310a-147641-0,0
1,amountreclassifiedfromaociaffecte line item th...,AmountreclassifiedfromAOCIAffected line items ...,2017-AAL-TR-AL-310a-147641-1,0
2,retiree medical postretirement benefit prior s...,retiree medical and other postretirement benef...,2017-AAL-TR-AL-310a-147641-2,0
0,earning share,Earnings per share:\n,2017-AAL-TR-AL-310a-147711-0,0
1,share computation,Shares used for computation (in thousands):\n,2017-AAL-TR-AL-310a-147711-1,0
2,result include total net special item principa...,Our fourth quarter 2017 results include $307 m...,2017-AAL-TR-AL-310a-147711-2,0
0,reclassification aoci follow,Reclassifications out of AOCI for the years en...,2017-AAL-TR-AL-310a-148045-0,0
1,amountreclassifiedfrom aociaffecte line item t...,Amountreclassifiedfrom AOCIAffected line items...,2017-AAL-TR-AL-310a-148045-1,0
2,end pension,"Ended December31,AOCI Components20172016Amorti...",2017-AAL-TR-AL-310a-148045-2,0
3,retiree medical postretirement benefit prior s...,retiree medical and other postretirement benef...,2017-AAL-TR-AL-310a-148045-3,0


## Step 3: Load processed data
The purpose of this section is to:
* Load the processed data (whichever the user chooses)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


# load lemmatized labeled data
df_300 = pd.read_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.7 visualized/labeled_data/di_hc_rel_train_lemmatized_no_ent_sent.csv')
# df_300.head(5)


---
# Heuristic identification and testing

## Step 1: Prepare data for analysis

The purpose of this section is to:
* Make a copy of the loaded dataframe containing the lemmatized excerpts in sentence form
* Convert the multi-class target into a binary-class target representing whether the records is tagged with GIC 310 or not
* Vectorize the lemmatized excerpts using tf-idf



In [ ]:
# make a copy for 330
df_330 = df_300.copy()

# remove nulls
df_330 = df_330[~(df_330['excerpt_lemma'].isnull())]

# mark 330 as relevant and everything else as 0
df_330['relevant'] = df_330['gic_code'].apply(lambda x: 1 if x == 330 else 0)

# confirm distribution
df_330['relevant'].value_counts()

0    67553
1    17936
Name: relevant, dtype: int64

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorize excerpts using tfidf
count_vec = TfidfVectorizer(stop_words='english', 
                            ngram_range=(1,3),
                            min_df=10)
X = df_330['excerpt_lemma']
y = df_330['relevant']
X = count_vec.fit_transform(X)
vocabs = dict(sorted(count_vec.vocabulary_.items(), key=lambda item: item[1]))
vocabs = [k for k, v in vocabs.items()]

## Step 2: Identify top and bottom n-grams correlated to target
The purpose of this section is to:
* Run a simple logistic regression over the dataset to determine the significance of each feature with respect to the targets
* Generate a bar chart showing the n-grams that are most correlated to the targets
* Generate a bar chart showing the n-grams that are least correlated to the targets

In [ ]:
from sklearn.linear_model import LogisticRegression

# run a simple logistic regression over the data
clf = LogisticRegression(class_weight='balanced', max_iter=100000)
clf.fit(X,y)

# generate a dataframe with n-gram and coefficients
df_coef = pd.DataFrame()
df_coef['n-gram'] = vocabs
df_coef['coefficient'] = clf.coef_[0]

In [ ]:
pip install chart_studio

     |████████████████████████████████| 71kB 4.6MB/s 


In [ ]:
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls
import plotly.graph_objects as go


# look at the top N n-grams by importance
n = 50
y_val = df_coef.sort_values(by='coefficient', ascending=False).head(n)['coefficient']
x_val = df_coef.sort_values(by='coefficient', ascending=False).head(n)['n-gram']

fig = go.Figure()
trace_bar_tmp = go.Bar(x=x_val, y=y_val, marker={'color': 'lightseagreen'})
fig.add_trace(trace_bar_tmp)
fig.update_layout(
    title='GIC 330: DEI: Top-'+str(n)+' n-grams by importance',
    template = 'seaborn',
)
fig.update_xaxes(tickangle=45, title_text='')
fig.update_yaxes(title_text='Coefficient')

fig.show()

In [ ]:
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls
import plotly.graph_objects as go

# look at the bottom N n-grams by importance
n = 50
y_val = df_coef.sort_values(by='coefficient', ascending=True).head(n)['coefficient']
x_val = df_coef.sort_values(by='coefficient', ascending=True).head(n)['n-gram']

fig = go.Figure()
trace_bar_tmp = go.Bar(x=x_val, y=y_val, marker={'color': 'lightsalmon'})
fig.add_trace(trace_bar_tmp)
fig.update_layout(
    title='GIC 330: DEI: Bottom-'+str(n)+' n-grams by importance',
    template = 'seaborn'
)
fig.update_xaxes(tickangle=45, title_text='')
fig.update_yaxes(title_text='Coefficient')

fig.show()

## Step 3: Cluster analysis to identify themes
The purpose of this section is to:
* Start the process by looking at only the sentences from paragraphs that are marked as relevant for GIC 310
* Run K-means clustering over the sentences to group them into similar topics by their tf-idf vectors
* For each cluster, review the sentences and identify the themes
* For each themes, come up with key words that we could use to come up with revised set of targets

The proposed rules are documented in this [Google Sheet](https://docs.google.com/spreadsheets/d/12zUx0bFlI4Q9HKks3VTFlXulA9rVOjqsbTCok3Mvs_Q/edit#gid=283194973&range=A1).

In [ ]:
# ref: https://www.kaggle.com/jbencina/clustering-documents-with-tfidf-and-kmeans
# attempt clustering on 330 excerpts

X = df_330[df_330['relevant'] == 1]['excerpt_lemma']
X = count_vec.transform(X)

# where is the clustering part?

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
import numpy as np

# run clustering for different k's to identify the optimal number of clusters
k = 40
sses = []
print('clustering k = ', end='')
for i in range(2,k+1):
  print(i, end=', ')
  fit_obj = MiniBatchKMeans(n_clusters=i, random_state=20)
  fit_obj.fit(X)
  
  sse = fit_obj.inertia_
  # sse = silhouette_score(X, fit_obj.labels_, metric='euclidean') # for silhouette score instead
  sses.append(sse)

y_val = sses
x_val = np.arange(2,k+1)

# graph results
import plotly.graph_objects as go

fig = go.Figure()
trace = go.Scatter(x=x_val, y=y_val, mode='lines+markers')
fig.add_trace(trace)
fig.update_layout(
    title='Sum of squared errors (SSE) by cluster size',
    template = 'seaborn'
)
fig.update_xaxes(title_text='Number of clusters')
fig.update_yaxes(title_text='SSE')

fig.show()

In [ ]:
from sklearn.cluster import MiniBatchKMeans
import numpy as np

# generate clusters based on the selected number of clusters

n_clusters = 28 # change this 
clusters = MiniBatchKMeans(n_clusters=n_clusters, random_state=20).fit_predict(X)

df_330_rel = df_330[df_330['relevant'] == 1]
df_330_rel['cluster'] = clusters
# df_330_rel['cluster'].value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
from collections import Counter

# review the n-grams with the highest tf-idf scores from each cluster

df_tfidf = pd.DataFrame()

for i in range(n_clusters):
  df_cluster = df_330_rel[df_330_rel['cluster'] == i]['excerpt_lemma']
  excerpts = np.asarray(count_vec.transform(df_cluster).sum(axis=0))[0]/df_cluster.shape[0]
  df_tmp = pd.DataFrame(zip(vocabs, excerpts), 
                          columns=['n-gram', 'norm_tfidf_score'])
  df_tmp['cluster'] = i
  df_tmp = df_tmp.sort_values(by='norm_tfidf_score', ascending=False)
  df_tfidf = pd.concat([df_tfidf, df_tmp.head(50)])

df_tfidf.to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.7 visualized/labeled_data/df_tfidf.csv', index=False)
df_tfidf.groupby('cluster').agg({'n-gram': ', '.join})

,n-gram
cluster,
0,"success depend, success, depend, ability, future success, depend ability, retain, future success depend, success depend ability, attract, personnel, future, ability attract, motivate, attract retain, technical, qualified, continue, retain motivate, highly, depend ability attract, ability attract retain, highly qualified, motivate highly, retain motivate highly, continue ability, attract retain motivate, skilled, management, depend continue, success depend continue, managerial, marketing, key, ability continue, retain highly, believe future, believe future success, qualified technical, ability continue attract, qualified personnel, employee, highly skilled, continue attract, sale, continue ability attract, highly qualified technical, retain highly qualified, continue attract retain, continue service"
1,"depend ability expand, strategy depend ability, growth strategy depend, strategy depend, ability expand, growth strategy, ability expand retain, retain organization, expand retain, expand retain organization, organization, expand, depend ability, strategy, growth, depend, world class talent, class talent, world class, ability, world, expand organization, talent, high quality personnel, quality personnel, class, retain, high quality, personnel, highly skilled personnel, skilled personnel, quality, highly skilled, high, hire, skilled, highly, permit approval require, period specify, period straight, period period, period present, permit development, period result, period rapid, permit conduct, permit approval, period rapid growth, permit authorization, period require"
2,"employee, time employee, time, locate, employee locate, employ, total, total employee, employee worldwide, worldwide, employee employee, employee time, employ time, time time, employee base, employ time employee, employee time employee, employee include, time employee time, temporary, time time employee, base, employ employee, outside, temporary employee, locate outside, total time, employee locate outside, employee respectively, locate locate, include, total time employee, respectively, employee locate locate, employment, employ time time, employee temporary, employment time, time employee include, include employee, employee temporary employee, grow employee employee, regular, grow employee, employee compare, compare, number, compare employee, time employee increase, employee increase"
3,"manage multiple brand, brand technology platform, multiple brand technology, multiple brand, platform acquire business, brand technology, technology platform acquire, acquire business increase, continue manage multiple, platform acquire, manage multiple, continue manage, business increase cost, technology platform, increase cost operation, cost operation, business increase, multiple, platform, acquire business, brand, increase cost, acquire, manage, technology, continue, case, increase, cost, operation, business, periodically, permanent employee, performance period, performance price, performance price common, performance product, permanent, periodically review, periodically experience, periodically evaluate, performance substantially, periodic report file, periodic pension, periodic inspection regulatory, periodic inspection, periodic delay preclude, periodic delay change, periodic delay, performance project"
4,"competition, intense, personnel, qualified, competition qualified, qualified personnel, industry, competition personnel, personnel intense, competition qualified personnel, particularly, employee intense, intense competition, retain, industry intense, competition employee, competition personnel intense, employee, personnel industry, intense particularly, technical, attract, competition employee intense, engineer, semiconductor, attract retain, successful, retain qualified, semiconductor industry, skilled, competition talent, competition skilled, high, intense competition qualified, individual, experienced, talent, attract retain qualified, successful

In [ ]:
# review the sentences within a selected cluster

df_330_rel[(df_330_rel['cluster'] == 0) \
          #  & (df_330_rel['excerpt'].str.contains('minimum wage'))\
          #  & ~(df_330_rel['excerpt'].str.contains('could|if|may|will|risk|potential|increased|decreased|expense|pension'))
           ][['excerpt', 'gic_code', 'cluster']].sample(5)

,excerpt,gic_code,cluster
70799,"Our success also depends upon retaining key management and technical personnel, as well as our ability to continue to attract and\nretain additional highly qualified personnel.",330,0
44646,"Planar‘s future success will depend largely\non its ability to continue to attract, retain, and motivate highly skilled and qualified personnel.",330,0
8138,"We believe that our success depends, and will likely continue to depend, upon our ability to retain the services of our current executive officers, directors, principal consultants and others.",330,0
41878,Our future success will depend in part upon our ability to attract and retain highly skilled and qualified personnel.,330,0
36295,"Our future success depends on our ability to hire and retain qualified management, marketing, finance, accounting and technical employees, including senior management.",330,0


## Step 4: Identify heuristics, apply to labeled dataset and review results

Based on the results from the previous steps, we are able to prepare [heuristics](https://docs.google.com/spreadsheets/d/1UFZXp4X_eqEtWtIRllvnBI3BlUbQBSMwWUJY4WTpSOQ/edit#gid=0&range=A1) that can be used to filter 10-K paragraphs based on the themes and keywords identified above as well as the [SASB materiality map](https://materiality.sasb.org/).

The purpose of this section is to:

* Apply heuristics to identify relevant 10-K paragraphs
* Generate one .csv containing all relevant paragraphs for downstream tasks (i.e. hypothesis testing) (saved in this [folder](https://drive.google.com/drive/u/1/folders/1FL4A61UZxGhqzKVWnikKNQ49weDsVLJh))
* Generate one .csv containing 100 sampled relevant paragraphs for manual review (saved in this [folder](https://drive.google.com/drive/u/1/folders/1FL4A61UZxGhqzKVWnikKNQ49weDsVLJh))
* Manually import and review the sampled dataset in [this Google Sheet](https://docs.google.com/spreadsheets/d/1UFZXp4X_eqEtWtIRllvnBI3BlUbQBSMwWUJY4WTpSOQ/edit?usp=sharing)
* Refine heuristics as needed

### Option 1: Use `pandas` to import data from csv and apply heuristics

Please note that:
* this option takes more time since it opens and loads the csv's for all years from Google Drive every single time
* we did not load all csv's at once because Colab will crash from running out of memory.

In [ ]:
def apply_heuristics(text, terms_to_include, terms_to_exclude):

  # lowercase text
  text = text.lower()

  # replace line breaks
  text = text.replace('\n', '')

  # space_symbol = '(\s|[!"\#$%&'+"'()*+,\-./:;<=>?@\[\\\]^_‘{|}~])"
  # terms_to_include = [t.replace(' ', space_symbol) for t in terms_to_include]
  # terms_to_exclude = [t.replace(' ', space_symbol) for t in terms_to_exclude]

  terms_to_include = terms_to_include[0] if len(terms_to_include) == 1 else '|'.join(terms_to_include)
  terms_to_exclude = terms_to_exclude[0] if len(terms_to_exclude) == 1 else '|'.join(terms_to_exclude)

  if (len(terms_to_include) == 0 or re.search(terms_to_include, text)) \
      and (len(terms_to_exclude) == 0 or (not re.search(terms_to_exclude, text))):
      return 1
  else:
    return 0

In [ ]:
#@title 330.1 { display-mode: "both" }
# import packages
import pandas as pd
import re
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


### UPDATE ME!!! SET UP COMPONENTS OF FILE NAME ###
GIC_HEURISTIC_ID = '330.1'

#### UPDATE ME!!! SET UP HEURISTICS ###
HEURISTICS = [
              # PASS #1
              {'include': [' women', ' woman', ' female', ' male'],
               'exclude': []},
              # PASS #2
              {'include': [],
               'exclude': [
                           'apparel', ' men\'s women\'s',
                           'apparel', ' women\'s and men\'s',
                           'apparel', ' women\s brands', ' men women'
                           ' women\'s sportswear', ' womenswear', ' fertility', ' egg', ' uterus',
                           ' infertility', 'menopaus', ' ovaries', ' embryo', ' IVF', ' bebe woman',
                           ' swimwear', ' fashion conscious woman', ' healthy female', ' healthy male',
                           ' reproductive', ' cancer', 'menstru', 'men and women', 'males and females',
                           ' potty', ' merchandise', 'males and females', ' decor', ' art', ' footwear',
                           ' accessories', ' death', ' disease', ' symptom', ' patient', ' infection',
                           ' shoes', ' cosmetics', ' designers'
                           ]},
              # PASS #3
              {'include': [],
               'exclude': [' blood', ' umbilical', ' wine',' deficiencies', ' multivitamin', ' doctor',
                           ' calori', ' estrogen', ' drug', ' dance', ' laxatives', ' smoking', ' Clinical',
                           'placebo', ' FDA', ' testosterone', ' covid', ' disorder', ' participants'
              ]}
              ]

# set up apply_heuristics method

# set up dataframes to store positive and negative results
df_result_pos = pd.DataFrame()
df_result_neg = pd.DataFrame()

# load 10k paragraphs and apply heuristics
for year in range(2013, 2022): # 10-k submission years

  # import each year's 10-K paragraphs
  df_tmp = pd.read_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.2 paragraphed/10ks_para_sics/10ks_para_sics_'+str(year)+'.csv')

  for h in HEURISTICS:
    # apply heuristics
    df_tmp['result'] = df_tmp['text'].progress_apply(lambda x: \
                          apply_heuristics(x, h['include'], h['exclude']))
    
    # keep some negative results for review later 
    df_tmp_neg = df_tmp[df_tmp['result'] == 0]
    if df_tmp_neg.shape[0] > 10:
      df_tmp_neg = df_tmp_neg.sample(10)
    df_result_neg = pd.concat((df_result_neg, df_tmp_neg))

    # remove all negative results
    df_tmp = df_tmp[df_tmp['result'] == 1]

   # append to positive results 
  df_result_pos = pd.concat((df_result_pos, df_tmp))

df_result_pos['gic_heuristic_id'] = GIC_HEURISTIC_ID
df_result_neg['gic_heuristic_id'] = GIC_HEURISTIC_ID

# check the shape of data
print('shape:', df_result_pos.shape)
print('shape:', df_result_neg.shape)

# export data
df_result_pos.to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_full_pos.csv', index=False)
df_result_pos[['gic_heuristic_id', 'id', 'ticker_display', 'primary_industry_id', 'text']].sample(100).to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_sample_pos.csv', index=False)
df_result_neg[['gic_heuristic_id', 'id', 'ticker_display', 'primary_industry_id', 'text']].to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_sample_neg.csv', index=False)


In [ ]:
#@title Default title text
# import packages
import pandas as pd
import re
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


### UPDATE ME!!! SET UP COMPONENTS OF FILE NAME ###
GIC_HEURISTIC_ID = '330.2'

#### UPDATE ME!!! SET UP HEURISTICS ###
HEURISTICS = [
              # PASS #1
              {'include': [' diverse', ' diversity',' minori', 'ethnicit','gender',
                           ],
               'exclude': []},
              # PASS #2
             
              {'include': [],
               'exclude': [' diversified customer base', ' investment', ' wildlife', 'diverse portfolio',
                           'diverse opportunities', ' diverse fuel', ' diverse mortgage', 'geographically',
                           ' patient', ' franchisees', ' financ']}
            
              ]


# set up dataframes to store positive and negative results
df_result_pos = pd.DataFrame()
df_result_neg = pd.DataFrame()

# load 10k paragraphs and apply heuristics
for year in range(2013, 2022): # 10-k submission years

  # import each year's 10-K paragraphs
  df_tmp = pd.read_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.2 paragraphed/10ks_para_sics/10ks_para_sics_'+str(year)+'.csv')

  for h in HEURISTICS:
    # apply heuristics
    df_tmp['result'] = df_tmp['text'].progress_apply(lambda x: \
                          apply_heuristics(x, h['include'], h['exclude']))
    
    # keep some negative results for review later 
    df_tmp_neg = df_tmp[df_tmp['result'] == 0]
    if df_tmp_neg.shape[0] > 10:
      df_tmp_neg = df_tmp_neg.sample(10)
    df_result_neg = pd.concat((df_result_neg, df_tmp_neg))

    # remove all negative results
    df_tmp = df_tmp[df_tmp['result'] == 1]

   # append to positive results 
  df_result_pos = pd.concat((df_result_pos, df_tmp))

df_result_pos['gic_heuristic_id'] = GIC_HEURISTIC_ID
df_result_neg['gic_heuristic_id'] = GIC_HEURISTIC_ID

# check the shape of data
print('shape:', df_result_pos.shape)
print('shape:', df_result_neg.shape)

# export data
df_result_pos.to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_full_pos.csv', index=False)
df_result_pos[['gic_heuristic_id', 'id', 'ticker_display', 'primary_industry_id', 'text']].sample(100).to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_sample_pos.csv', index=False)
df_result_neg[['gic_heuristic_id', 'id', 'ticker_display', 'primary_industry_id', 'text']].to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_sample_neg.csv', index=False)


In [ ]:
#@title Default title text
# import packages
import pandas as pd
import re
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

### UPDATE ME!!! SET UP COMPONENTS OF FILE NAME ###
GIC_HEURISTIC_ID = '330.3'

#### UPDATE ME!!! SET UP HEURISTICS ###
HEURISTICS = [
              # PASS #1
              {'include': [' race',' racism',' racial',' discriminat'],
               'exclude': []},
              # PASS #2
              # {'include': [],
              #  'exclude': ['racetrack', 'black-scholes', 'blackhawk', 'black hills', 'black tattoos',
              #              'black market', 'white wolf', 'whitegate', 'black bird', 'spartan race inc', 'blackstone',
              #              'blackwell', 'blackout', 'black lake', 'white water rafting', 'white blood cells', 'white furnishings']},
              
              {'include': [],
               'exclude': ['racetrack', 'spartan race inc' ' racer', 'formula 1', ' circuit', ' driver', ' raceland',
                           'nascar', 'race promot', 'race teams', 'raceway', 'discrimination in credit', ' race park', 'horse',
                           'track', 'racemic', 'racing']}
              
              ]


# set up dataframes to store positive and negative results
df_result_pos = pd.DataFrame()
df_result_neg = pd.DataFrame()

# load 10k paragraphs and apply heuristics
for year in range(2013, 2022): # 10-k submission years

  # import each year's 10-K paragraphs
  df_tmp = pd.read_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.2 paragraphed/10ks_para_sics/10ks_para_sics_'+str(year)+'.csv')

  for h in HEURISTICS:
    # apply heuristics
    df_tmp['result'] = df_tmp['text'].progress_apply(lambda x: \
                          apply_heuristics(x, h['include'], h['exclude']))
    
    # keep some negative results for review later 
    df_tmp_neg = df_tmp[df_tmp['result'] == 0]
    if df_tmp_neg.shape[0] > 10:
      df_tmp_neg = df_tmp_neg.sample(10)
    df_result_neg = pd.concat((df_result_neg, df_tmp_neg))

    # remove all negative results
    df_tmp = df_tmp[df_tmp['result'] == 1]

   # append to positive results 
  df_result_pos = pd.concat((df_result_pos, df_tmp))

df_result_pos['gic_heuristic_id'] = GIC_HEURISTIC_ID
df_result_neg['gic_heuristic_id'] = GIC_HEURISTIC_ID

# check the shape of data
print('shape:', df_result_pos.shape)
print('shape:', df_result_neg.shape)

# export data
df_result_pos.to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_full_pos.csv', index=False)
df_result_pos[['gic_heuristic_id', 'id', 'ticker_display', 'primary_industry_id', 'text']].sample(100).to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_sample_pos.csv', index=False)
df_result_neg[['gic_heuristic_id', 'id', 'ticker_display', 'primary_industry_id', 'text']].to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_sample_neg.csv', index=False)


In [ ]:
#@title 330.4
# import packages
import pandas as pd
import re
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

### UPDATE ME!!! SET UP COMPONENTS OF FILE NAME ###
GIC_HEURISTIC_ID = '330.4'

#### UPDATE ME!!! SET UP HEURISTICS ###
HEURISTICS = [
              # PASS #1
              {'include': [' hostile',' bully',' harass', ' sexual', ' inappropriate'],
               'exclude': []}
              ]


# set up dataframes to store positive and negative results
df_result_pos = pd.DataFrame()
df_result_neg = pd.DataFrame()

# load 10k paragraphs and apply heuristics
for year in range(2013, 2022): # 10-k submission years

  # import each year's 10-K paragraphs
  df_tmp = pd.read_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.2 paragraphed/10ks_para_sics/10ks_para_sics_'+str(year)+'.csv')

  for h in HEURISTICS:
    # apply heuristics
    df_tmp['result'] = df_tmp['text'].progress_apply(lambda x: \
                          apply_heuristics(x, h['include'], h['exclude']))
    
    # keep some negative results for review later 
    df_tmp_neg = df_tmp[df_tmp['result'] == 0]
    if df_tmp_neg.shape[0] > 10:
      df_tmp_neg = df_tmp_neg.sample(10)
    df_result_neg = pd.concat((df_result_neg, df_tmp_neg))

    # remove all negative results
    df_tmp = df_tmp[df_tmp['result'] == 1]

   # append to positive results 
  df_result_pos = pd.concat((df_result_pos, df_tmp))

df_result_pos['gic_heuristic_id'] = GIC_HEURISTIC_ID
df_result_neg['gic_heuristic_id'] = GIC_HEURISTIC_ID

# check the shape of data
print('shape:', df_result_pos.shape)
print('shape:', df_result_neg.shape)

# export data
df_result_pos.to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_full_pos.csv', index=False)
df_result_pos[['gic_heuristic_id', 'id', 'ticker_display', 'primary_industry_id', 'text']].sample(100).to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_sample_pos.csv', index=False)
df_result_neg[['gic_heuristic_id', 'id', 'ticker_display', 'primary_industry_id', 'text']].to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_sample_neg.csv', index=False)


###Option 2: Use Google BigQuery to perform the keyword search

Please note that:
* this option is SIGNIFICANTLY faster since a table with data from all submission years (2013-2021) is already stored on BigQuery
* there is a monthly cost associated with the storage of the data (first 10GB is free) as well as querying of the data (first 1TB of query is free)
* BigQuery can be accessed [here](https://console.cloud.google.com/bigquery?authuser=1&project=dfg-cohrv-314114&ws=!1m5!1m4!4m3!1sdfg-cohrv-314114!2sdfg_cohrv!3s10ks_para_sics&d=dfg_cohrv&p=dfg-cohrv-314114&t=10ks_para_sics&page=table) where you can query the data directly from the browser and export the result into a Google Sheet for further analysis

In [ ]:
#@title Default title text
# import packages
import pandas as pd
import re
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

### UPDATE ME!!! SET UP COMPONENTS OF FILE NAME ###
GIC_HEURISTIC_ID = '330.5'

#### UPDATE ME!!! SET UP HEURISTICS ###
hiring_context = '( hire |hiring|hired)'
women_minorities_context = '(women|female|minorities| color |enthinicities)' 
any_valid_chars = '([A-Za-z\d%,;\'\\""""\s])*'

HEURISTICS = [
              # PASS #1
              {'include': ['('+any_valid_chars+hiring_context+any_valid_chars\
                    +women_minorities_context+any_valid_chars+'[.?!])'],
               'exclude': []}
              ]


# set up dataframes to store positive and negative results
df_result_pos = pd.DataFrame()
df_result_neg = pd.DataFrame()

# load 10k paragraphs and apply heuristics
for year in range(2013, 2022): # 10-k submission years

  # import each year's 10-K paragraphs
  df_tmp = pd.read_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.2 paragraphed/10ks_para_sics/10ks_para_sics_'+str(year)+'.csv')

  for h in HEURISTICS:
    # apply heuristics
    df_tmp['result'] = df_tmp['text'].progress_apply(lambda x: \
                          apply_heuristics(x, h['include'], h['exclude']))
    
    # keep some negative results for review later 
    df_tmp_neg = df_tmp[df_tmp['result'] == 0]
    if df_tmp_neg.shape[0] > 10:
      df_tmp_neg = df_tmp_neg.sample(10)
    df_result_neg = pd.concat((df_result_neg, df_tmp_neg))

    # remove all negative results
    df_tmp = df_tmp[df_tmp['result'] == 1]

   # append to positive results 
  df_result_pos = pd.concat((df_result_pos, df_tmp))

df_result_pos['gic_heuristic_id'] = GIC_HEURISTIC_ID
df_result_neg['gic_heuristic_id'] = GIC_HEURISTIC_ID

# check the shape of data
print('shape:', df_result_pos.shape)
print('shape:', df_result_neg.shape)

# export data
df_result_pos.to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_full_pos.csv', index=False)
df_result_pos[['gic_heuristic_id', 'id', 'ticker_display', 'primary_industry_id', 'text']].sample(100).to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_sample_pos.csv', index=False)
df_result_neg[['gic_heuristic_id', 'id', 'ticker_display', 'primary_industry_id', 'text']].to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_sample_neg.csv', index=False)


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version



In [ ]:
# authenticate to BigQuery
# if you do not have access, please contact Lawrence at lcl2152@columbia.edu

from google.colab import auth
auth.authenticate_user()

project_id = 'dfg-cohrv-314114'

In [ ]:
# preprocessing to append data to the BigQuery table
# DO NOT RERUN. THIS STEP IS ALREADY DONE FOR 2013-2021 SUBMISSION YEARS

# import pandas as pd

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
# project_id = 'dfg-cohrv-314114'
# for year in range(2013,2022):
#   df_tmp = pd.read_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.2 paragraphed/10ks_para_sics/10ks_para_sics_'+str(year)+'.csv')
#   df_tmp.to_gbq('dfg_cohrv.10ks_para_sics', project_id, if_exists='append', chunksize=100000)

In [ ]:
#@title Irrelevant
import pandas as pd
# https://cloud.google.com/resource-manager/docs/creating-managing-projects

##### UPDATE ME! #####
GIC_HEURISTIC_ID = '310.2' 

# don't worry about case sensitivity. it is taken care of in the SQL statement

terms_to_include = [' collective bargain', ' cba']

# terms_to_exclude = []

terms_to_exclude = [' no collective bargain', ' any collective bargain',
                    ' none of ', ' not covered ', ' not represented ', 
                    ' not subject to ', ' no \w+ covered ', 
                    ' no \w+ represented ', ' not a party ', ' not parties ']

# combine terms to include into a regex statement
if len(terms_to_include) > 0:
  terms_to_include = terms_to_include[0] if len(terms_to_include) == 1 else '|'.join(terms_to_include)
else:
  terms_to_include = '~'

# combine terms to exclude into a regex statement
if len(terms_to_exclude) > 0:  
  terms_to_exclude = terms_to_exclude[0] if len(terms_to_exclude) == 1 else '|'.join(terms_to_exclude)
else:
  terms_to_exclude = '~'

# retrieve positive results
df_result_pos = pd.io.gbq.read_gbq(
  '''
  SELECT *
  FROM `dfg_cohrv.10ks_para_sics`
  WHERE 
    REGEXP_CONTAINS(text, r"(?i)('''+terms_to_include+''')") 
    AND NOT REGEXP_CONTAINS(text, r"(?i)('''+terms_to_exclude+''')")
  '''
, project_id=project_id)

# retrieve negative results
df_result_neg = pd.io.gbq.read_gbq(
  '''
  WITH t AS
  (
    SELECT *
    FROM `dfg_cohrv.10ks_para_sics`
    WHERE 
      REGEXP_CONTAINS(text, r"(?i)('''+terms_to_include+''')") 
      AND REGEXP_CONTAINS(text, r"(?i)('''+terms_to_exclude+''')")
  )
  SELECT * FROM t
  WHERE RAND() < 100/(SELECT COUNT(*) FROM t)  
  '''
, project_id=project_id)

df_result_pos['gic_heuristic_id'] = GIC_HEURISTIC_ID
df_result_neg['gic_heuristic_id'] = GIC_HEURISTIC_ID

df_result_pos = df_result_pos[['gic_heuristic_id', 'id', 'ticker_display', 'primary_industry_id', 'text']]
df_result_neg = df_result_neg[['gic_heuristic_id', 'id', 'ticker_display', 'primary_industry_id', 'text']]

df_result_pos.to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_full_pos.csv', index=False)
df_result_pos.sample(100).to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_sample_pos.csv', index=False)
df_result_neg.to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/10ks_para_sics_gic'+GIC_HEURISTIC_ID+'_sample_neg.csv', index=False)

Use the following cell to review specific data

In [ ]:
#@title Irrelevant
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


GIC_HEURISTIC_ID = '310.2' 

terms_to_include = ['\s*collective bargain', ' cba']

# terms_to_exclude = []

terms_to_exclude = ['\s*(no|any) collective bargain',
                    '\s*(no|none|not)+(\w|\s|\')*(cover|subject|represent)+(\w|\s|\')*collective bargain', 
                    '\s*collective bargain(\w|\s|\')*do\w* not cover', 
                    '\s*do\w* not have (\w|\s|\')*collective bargain',
                    '\s*not a*\s*(party|parties)+(\w|\s|\')*collective bargain',
                    ' discount']

# combine terms to include into a regex statement
if len(terms_to_include) > 0:
  terms_to_include = terms_to_include[0] if len(terms_to_include) == 1 else '|'.join(terms_to_include)
else:
  terms_to_include = '~'

# combine terms to exclude into a regex statement
if len(terms_to_exclude) > 0:  
  terms_to_exclude = terms_to_exclude[0] if len(terms_to_exclude) == 1 else '|'.join(terms_to_exclude)
else:
  terms_to_exclude = '~'

df_tmp = pd.io.gbq.read_gbq(
  '''
  WITH t AS
  (
    SELECT *
    FROM `dfg_cohrv.10ks_para_sics`
    WHERE 
      REGEXP_CONTAINS(text, r"(?i)('''+terms_to_include+''')") 
      AND REGEXP_CONTAINS(text, r"(?i)('''+terms_to_exclude+''')")
      AND REGEXP_CONTAINS(id, r"_2013")
      # AND primary_industry_id = "CG-AA"
  )
  SELECT * FROM t
  '''
, project_id=project_id)

df_tmp['gic_heuristic_id'] = GIC_HEURISTIC_ID
df_tmp = df_tmp[['gic_heuristic_id', 'id', 'ticker_display', 'primary_industry_id', 'text']]
df_tmp = df_tmp.sort_values(by=['primary_industry_id', 'ticker_display'])
# df_tmp.to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_1.0 hypothesis testing/df_tmp.csv', index=False)
df_tmp[['primary_industry_id', 'ticker_display', 'text']]


In [ ]:
len(df_tmp['ticker_display'].unique())

---
# DO NOT USE

In [ ]:
CHECKED_TICKERS = ['AAL', 'AAWW', 'ALSK', 'APG', 'ATSG', 'BXC', 'EME', 'EXC', 'FE', 'HA', 'HCC', 'HMTV', 'IEA', 'INTG', 'LMB', 'MSGN', 'PRSI', 'SCS', 'SFEF']

df_result_pos[(df_result_pos['text'].str.lower().str.contains(' cba '))
              & ~(df_result_pos['text'].str.lower().str.contains(' collective bargain'))
              & ~(df_result_pos['text'].str.lower().str.contains('pension|retirement'))
              & ~(df_result_pos['ticker_display'].isin(CHECKED_TICKERS))
              ][['ticker_display', 'text']].sort_values(by='ticker_display')

In [ ]:
# make a copy of the original dataframe

df_310_test = df_300.reset_index(drop=True).copy()

In [ ]:
import re
from itertools import permutations 
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas(desc="progress bar")


def apply_gic_310_heuristics(text):

  # lowercase text
  text = text.lower()

  # replace line breaks
  text = text.replace('\n', '')

  # check for short text (remove anything less than 10 words)
  if text.count(' ') < 2:
    return 0

  # prepare lawsuit related terms
  lawsuit_terms = ['lawsuit', 'litigation', 'case', 'court', 'claim', 'ruling',
                   'proceeding', 'appeal', 'judgment']
  lawsuit_topics = ['harrassment', 'discrimination']

  lawsuit_include = []
  for term in lawsuit_terms:
    for topic in lawsuit_topics:
      lawsuit_include.append(term+'.*'+topic)
      lawsuit_include.append(topic+'.*'+term)


  # apply heuristics
  gic_310_heuristics = [                           
                        # unions
                        {'include': ['union', 'organized labor', 'labor organization', 
                                      'labor association', 'recognition agreement',
                                     'labor agreement'], 
                          'exclude': ['relation', 
                                      'satisfactory', 'good', 
                                      'european union', 'no', 'none',
                                      'union city', 'city of union',
                                      'town of union']},
                        
                        # labor dispute
                        {'include': ['stoppage', 'strike', 'labor dispute', 
                                      'labor disagreement', 'labor disruption'], 
                          'exclude': ['strike price', 'lightning strike',
                                      'equipment failure', 'disaster',
                                      'malfunction', 'defect', 'decay',
                                      'inadequacy', 'repair', 'decoration',
                                      'choking', 'bursting', 'overflow', 
                                      'leakage', 'hazard', 'breakdown']},
                        
                        # collective bargaining
                        {'include': ['collective bargain', 'CBA'], 
                          'exclude': ['no', 'none']},
                      
                        # independent contractor
                        {'include': ['independent contractor'], 
                          'exclude': []},
                      
                        # recruitment practice
                        {'include': ['recruit.*practice', 'manning agent',
                                      'crewing', 'crewed'], 
                          'exclude': []},
                      
                        # workers' compensation
                        {'include': ["workers' compensation"], 
                          'exclude': []},
                      
                        # unauthorized worker
                        {'include': ['unauthorized worker'], 
                          'exclude': []},
                        
                        # overtime
                          {'include': ['overtime'], 
                            'exclude': []},
                        
                        # minimum wage
                        {'include': ['exceed.*minimum wage', 'greater.*minimum wage', 
                                      'lower.*minimum wage', 'less.*minimum wage',
                                      'higher.*minimum wage', 'tip credit',
                                      'tipped employee'], 
                          'exclude': []},
                        
                      
                        # lawsuits (refer to above)
                        {'include': lawsuit_include,
                          'exclude': []},
                        
  ]

  for h in gic_310_heuristics:
    terms_to_include = h['include'][0] if len(h['include']) == 1 else '|'.join(h['include'])
    terms_to_exclude = h['exclude'][0] if len(h['exclude']) == 1 else '|'.join(h['exclude'])
    # print('terms_to_include:', terms_to_include)
    # print('terms_to_exclude:', terms_to_exclude)
    if (len(terms_to_include) == 0 or re.search(terms_to_include, text)) \
      and (len(terms_to_exclude) == 0 or (not re.search(terms_to_exclude, text))):
      return 310
  return 0
    
df_310_test['new_gic_code'] = df_310_test['excerpt']\
  .progress_apply(lambda x: apply_gic_310_heuristics(x))

df_310_test[\
            (df_310_test['gic_code'] != df_310_test['new_gic_code'])\
            & ((df_310_test['gic_code'] == 310)|(df_310_test['new_gic_code'] == 310))\
            ][['excerpt_id', 'excerpt', 'gic_code', 'new_gic_code']]
# .to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.7 visualized/labeled_data/df_310_test_diffs_v3.csv', index=False)

In [ ]:
# df_310_test[(df_310_test['gic_code'] != df_310_test['new_gic_code'])
#             & df_310_test['excerpt'].str.contains('collective bargain')].head(50)

df_310_test[\
            # (df_310_test['gic_code'] != df_310_test['new_gic_code'])\
            # & ((df_310_test['gic_code'] == 310)|(df_310_test['new_gic_code'] == 310))\
            # (df_310_test['excerpt'].str.contains('CBA'))\
            (df_310_test['excerpt_id'].str.contains('A1_M_CMP_10-K_EM-MM-310a_56343'))\
            # & (df_310_test['excerpt'].str.contains('exceed.*minimum wage'))\
  ][['excerpt_id', 'excerpt', 'gic_code', 'new_gic_code']]
  # .to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.7 visualized/labeled_data/df_310_test_diffs.csv',
  #                                                                index=False)

---
# DO NOT USE

In [ ]:
# df_300 = df_300.reset_index(drop=True)

# terms_to_exclude = '\$|increased|decreased|expense|record|'\
#                     +'if|could|may|would|will|can|'\
#                     +'factor|ability to|risk|future|potential|expect|'\
#                     +'atract|retain|qualified|'\
#                     +'defined|benefit|contribution|retirement|profit sharing|'\
#                     +'award|ranking|'\
#                     +'strike price|strike out|insurance|breakdown|'\
#                     +'must|subject|govern|adopt|regulat|appl|example'

terms_to_include =  'recruit'
                    # 'plaintiff|litigation|lawsuit|ruling|proceeding|court|appeal|claim'
                    # 'minimum wage'
                    # 'part time|part-time|hourly.*employee|temporary.*employee|constract.*employee'
                    #'stoppage|interruption|strike|labor dispute|labor disagreement'
                    #'union|unionize|organized labor|labor organization'
                    # +'collective bargain|'\

# df_test = df_300[(~(df_300['excerpt'].str.contains(terms_to_exclude)))
#                  & (df_300['excerpt'].str.contains(terms_to_include))]
df_test
# print(df_test['gic_code'].shape)
# df_test['gic_code'].value_counts()

---
## Investigate GIC 310 (using paragraphs)

In [ ]:
# make a copy for 310
df_310 = df_300.copy()

# remove nulls
df_310 = df_310[~df_310['excerpt_lemma'].isnull()]

# mark 310 as relevant and everything else as 0
df_310['relevant'] = df_310['gic_code'].apply(lambda x: 1 if x == 310 else 0)

# confirm distribution
df_310['relevant'].value_counts()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorize excerpts using tfidf
count_vec = TfidfVectorizer(stop_words='english', 
                            ngram_range=(1,3),
                            min_df=10)
X = df_310['excerpt_lemma']
y = df_310['relevant']
X = count_vec.fit_transform(X)
vocabs = dict(sorted(count_vec.vocabulary_.items(), key=lambda item: item[1]))
vocabs = [k for k, v in vocabs.items()]

In [ ]:
from sklearn.linear_model import LogisticRegression

# run a simple logistic regression over the data
clf = LogisticRegression(class_weight='balanced', max_iter=100000)
clf.fit(X,y)

In [ ]:
# generate a dataframe with n-gram and coefficients

df_coef = pd.DataFrame()
df_coef['n-gram'] = vocabs
df_coef['coefficient'] = clf.coef_[0]

In [ ]:
pip install chart_studio

In [ ]:
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls
import plotly.graph_objects as go


# look at the top N n-grams by importance
n = 50
y_val = df_coef.sort_values(by='coefficient', ascending=False).head(n)['coefficient']
x_val = df_coef.sort_values(by='coefficient', ascending=False).head(n)['n-gram']

fig = go.Figure()
trace_bar_tmp = go.Bar(x=x_val, y=y_val, marker={'color': 'lightseagreen'})
fig.add_trace(trace_bar_tmp)
fig.update_layout(
    title='GIC 310: Labor Practice: Top-'+str(n)+' n-grams by importance',
    template = 'seaborn',
)
fig.update_xaxes(tickangle=45, title_text='')
fig.update_yaxes(title_text='Coefficient')


fig.show()

In [ ]:
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls
import plotly.graph_objects as go

# look at the bottom N n-grams by importance
n = 50
y_val = df_coef.sort_values(by='coefficient', ascending=True).head(n)['coefficient']
x_val = df_coef.sort_values(by='coefficient', ascending=True).head(n)['n-gram']

fig = go.Figure()
trace_bar_tmp = go.Bar(x=x_val, y=y_val, marker={'color': 'lightsalmon'})
fig.add_trace(trace_bar_tmp)
fig.update_layout(
    title='GIC 310: Labor Practice: Bottom-'+str(n)+' n-grams by importance',
    template = 'seaborn'
)
fig.update_xaxes(tickangle=45, title_text='')
fig.update_yaxes(title_text='Coefficient')

fig.show()

In [ ]:
# ref: https://www.kaggle.com/jbencina/clustering-documents-with-tfidf-and-kmeans

# attempt clustering on 310 excerpts

X = df_310[df_310['relevant'] == 1]['excerpt_lemma']
X = count_vec.transform(X)

In [ ]:
from sklearn.cluster import MiniBatchKMeans
import numpy as np

# run clustering for different k's
k = 50
sses = []
print('clustering k = ', end='')
for i in range(2,k+1):
  print(i, end=', ')
  sse = MiniBatchKMeans(n_clusters=i, init_size=1024, 
                        batch_size=2048, random_state=20).fit(X).inertia_
  sses.append(sse)

y_val = sses
x_val = np.arange(2,k+1)

# graph results
import plotly.graph_objects as go

fig = go.Figure()
trace = go.Scatter(x=x_val, y=y_val, mode='lines+markers')
fig.add_trace(trace)
fig.update_layout(
    title='Sum of squared errors (SSE) by cluster size',
    template = 'seaborn'
)
fig.update_xaxes(title_text='Number of clusters')
fig.update_yaxes(title_text='SSE')

fig.show()

In [ ]:
# Generate clusters based on the selected number of clusters

n_clusters = 14
clusters = MiniBatchKMeans(n_clusters=n_clusters, init_size=1024, 
                           batch_size=2048, random_state=20).fit_predict(X)

df_310_rel = df_310[df_310['relevant'] == 1]
df_310_rel['cluster'] = clusters
df_310_rel['cluster'].value_counts()

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
# from wordcloud import WordCloud, STOPWORDS
# import scipy

df_tfidf = pd.DataFrame()

for i in range(n_clusters):
  df_cluster = df_310_rel[df_310_rel['cluster'] == i]['excerpt_lemma']
  excerpts = np.asarray(count_vec.transform(df_cluster).sum(axis=0))[0]/df_cluster.shape[0]
  df_tmp = pd.DataFrame(zip(vocabs, excerpts), 
                          columns=['n-gram', 'norm_tfidf_score'])
  df_tmp['cluster'] = i
  df_tmp = df_tmp.sort_values(by='norm_tfidf_score', ascending=False)
  df_tfidf = pd.concat([df_tfidf, df_tmp.head(20)])

df_tfidf.to_csv('/content/drive/MyDrive/DFG Cost of Human Rights Violations/Datasets/10k_clean_text/_0.7 visualized/labeled_data/df_tfidf.csv', index=False)
df_tfidf.groupby('cluster').agg({'n-gram': ', '.join})

In [ ]:
df_310_rel[(df_310_rel['cluster'] == 12) \
          #  & (df_310_rel['excerpt'].str.contains('stoppage'))
           ][['excerpt', 'gic_code', 'cluster']].head(20)